In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
file_list = [f for f in os.listdir('/Users/zihangwei/Library/CloudStorage/OneDrive-TexasA&MUniversity/Publishing_Paper/PDE_Network_Modeling/2_RawData/District6') if f.endswith('.txt')]

In [3]:
delimiter = ','  # Change this to the actual delimiter if it's different
data_all = pd.DataFrame()
for file_path in file_list:
    # Read the text file into a Pandas DataFrame
    df = pd.read_csv('/Users/zihangwei/Library/CloudStorage/OneDrive-TexasA&MUniversity/Publishing_Paper/PDE_Network_Modeling/2_RawData/District6/' + file_path, delimiter=delimiter, header=None)
    df = df.iloc[:, :12]
    names = ['TimeStamp', 'Station', 'District', 'Freeway_#', 'Dir_Travel', 'Type', 'Length', 'Samples',
             'Observed_%', 'Total_Flow', 'Occupancy', 'Speed']
    df.columns = names
    df = df[df['Type']=="ML"]

    data_all = pd.concat([data_all, df], ignore_index=True)

In [4]:
data_all_observed = data_all[data_all['Observed_%'] > 0]
summary_observed = data_all_observed.groupby('Station').size().reset_index(name='Count')

In [5]:
metadata = pd.read_csv('Data_Raw/Archive/d06_text_meta_2023_08_08.txt', delimiter='\t')
metadata = metadata[metadata['Type']=="ML"]

In [6]:
metadata_obs = pd.merge(metadata, summary_observed, left_on='ID', right_on='Station' , how='left')

In [7]:
data_all_observed

,TimeStamp,Station,District,Freeway_#,Dir_Travel,Type,Length,Samples,Observed_%,Total_Flow,Occupancy,Speed
0,08/20/2023 00:00:00,601100,6,180,W,ML,0.205,10,100,4.0,0.0033,65.0
1,08/20/2023 00:00:00,601101,6,41,S,ML,0.436,19,100,32.0,0.0140,67.2
2,08/20/2023 00:00:00,601102,6,41,S,ML,2.585,16,100,26.0,0.0094,68.1
3,08/20/2023 00:00:00,601103,6,180,E,ML,0.460,20,100,20.0,0.0075,68.5
4,08/20/2023 00:00:00,601105,6,99,N,ML,0.620,30,100,89.0,0.0225,68.7
...,...,...,...,...,...,...,...,...,...,...,...,...
1237817,08/15/2023 23:55:00,629977,6,99,N,ML,1.711,27,33,61.0,0.0171,68.1
1237819,08/15/2023 23:55:00,629989,6,99,N,ML,0.632,27,100,26.0,0.0074,66.2
1237820,08/15/2023 23:55:00,629990,6,99,S,ML,1.269,27,100,61.0,0.0171,65.5
1237821,08/15/2023 23:55:00,629991,6,5,S,ML,2.668,18,100,30.0,0.0122,67.4


## Station Selection

In [7]:
metadata_obs.to_csv('Data_Raw/Station_MetaData_d06.csv', index=False)

In [8]:
station_selected = pd.read_csv('Data_Raw/I5N_Exported.csv')
station_selected = station_selected[station_selected['Count'] == 2016]

In [9]:
station_selected['PM_Dir'] = station_selected['Abs_PM'] - station_selected['Abs_PM'].min()
#station_selected['PM_Dir_round0.5'] = station_selected['PM_Dir'].apply(lambda x: round(x * 2) / 2)
#station_selected['Station_PM'] = station_selected['PM_Dir'].round(0)

station_selected['Station_PM'] = np.round(station_selected['PM_Dir'] / 2) * 2
station_selected['PM_Deviation'] = abs(station_selected['PM_Dir'] - station_selected['Station_PM'])

In [10]:
# select the stations where the deviation from integer mile is the smallest.
min_index = station_selected.groupby('Station_PM')['PM_Deviation'].idxmin()
station_selected = station_selected.loc[min_index]

In [11]:
station_selected['location_step'] = station_selected['Station_PM']/2
station_selected['Station_Number'] = station_selected['Station_PM'].rank(ascending=True)

In [12]:
# select the first 60 miles of I-5 North
station_selected = station_selected[station_selected['Station_PM'] < 60]

In [13]:
station_selected

,ID,Fwy,Dir,District,County,City,State_PM,Abs_PM,Latitude,Longitude,...,User_ID_2,User_ID_3,User_ID_4,Station,Count,PM_Dir,Station_PM,PM_Deviation,location_step,Station_Number
12,601286,5,N,6,29,NaN,8.331,213.448,34.914506,-118.923609,...,NaN,NaN,NaN,601286.0,2016.0,0.000,0.0,0.000,0.0,1.0
37,601510,5,N,6,29,NaN,10.173,215.290,34.939412,-118.929744,...,NaN,NaN,NaN,601510.0,2016.0,1.842,2.0,0.158,1.0,2.0
22,601339,5,N,6,29,NaN,13.993,219.110,34.993067,-118.947281,...,NaN,NaN,NaN,601339.0,2016.0,5.662,6.0,0.338,3.0,3.0
10,601284,5,N,6,29,NaN,16.174,221.291,35.022381,-118.960142,...,NaN,NaN,NaN,601284.0,2016.0,7.843,8.0,0.157,4.0,4.0
7,601281,5,N,6,29,NaN,29.866,234.983,35.170050,-119.120044,...,NaN,NaN,NaN,601281.0,2016.0,21.535,22.0,0.465,11.0,5.0
25,601357,5,N,6,29,NaN,32.072,237.189,35.193824,-119.146126,...,NaN,NaN,NaN,601357.0,2016.0,23.741,24.0,0.259,12.0,6.0
14,601310,5,N,6,29,NaN,39.168,244.285,35.271319,-119.228751,...,NaN,NaN,NaN,601310.0,2016.0,30.837,30.0,0.837,15.0,7.0
38,601516,5,N,6,29,NaN,43.927,249.044,35.320704,-119.287451,...,NaN,NaN,NaN,601516.0,2016.0,35.596,36.0,0.404,18.0,8.0
39,601518,5,N,6,29,NaN,48.642,253.759,35.365470,-119.350584,...,NaN,NaN,NaN,601518.0,2016.0,40.311,40.0,0.311,20.0,9.0
6,601280,5,N,6,29,NaN,52.570,257.687,35.403061,-119.402843,...,NaN,NaN,NaN,601280.0,2016.0,44.239,44.0,0.239,22.0,10.0


In [14]:
station_selected.to_csv('Data_Raw/Station_Selected_I5N.csv', index=False)

In [14]:
order_match = station_selected[['ID', 'location_step']]
order_match = order_match.rename(columns={'location_step': 'Station_PM'})

In [15]:
ID_selected = station_selected['ID']
data_all_select = data_all[data_all['Station'].isin(ID_selected)]
data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
data_all_select['TimeStep'] = 1 + (data_all_select['TimeDiff'] // 5)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_13582/440614197.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeStamp'] = pd.to_datetime(data_all_select['TimeStamp'])
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_13582/440614197.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_all_select['TimeDiff'] = ((data_all_select['TimeStamp'] - data_all_select['TimeStamp'].min()).dt.total_seconds())/60
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_1

In [16]:
#station_selected[station_selected['PM_Dir_round'].duplicated(keep=False)]

In [17]:
data_all_select = pd.merge(data_all_select, order_match, left_on='Station', right_on='ID' , how='left')

In [18]:
Flow = data_all_select[['TimeStep', 'Station_PM', 'Total_Flow']]
Flow['TimeStep'] = Flow['TimeStep'].astype(int)
Flow['Station_PM'] = Flow['Station_PM'].astype(int)

Occupancy = data_all_select[['TimeStep', 'Station_PM', 'Occupancy']]
Occupancy['TimeStep'] = Occupancy['TimeStep'].astype(int)
Occupancy['Station_PM'] = Occupancy['Station_PM'].astype(int)

Speed = data_all_select[['TimeStep', 'Station_PM', 'Speed']]
Speed['TimeStep'] = Speed['TimeStep'].astype(int)
Speed['Station_PM'] = Speed['Station_PM'].astype(int)

/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_13582/3535391722.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['TimeStep'] = Flow['TimeStep'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_13582/3535391722.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Flow['Station_PM'] = Flow['Station_PM'].astype(int)
/var/folders/g5/yg5rt0h52kxc5z_0zd7jrqbh0000gn/T/ipykernel_13582/3535391722.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [20]:
data_all_select

,TimeStamp,Station,District,Freeway_#,Dir_Travel,Type,Length,Samples,Observed_%,Total_Flow,Occupancy,Speed,TimeDiff,TimeStep,ID,Station_PM
0,2023-08-20 00:00:00,601241,6,5,N,ML,2.060,20,100,31.0,0.0165,67.7,7200.0,1441.0,601241,26.0
1,2023-08-20 00:00:00,601279,6,5,N,ML,2.020,20,100,30.0,0.0158,67.7,7200.0,1441.0,601279,27.0
2,2023-08-20 00:00:00,601280,6,5,N,ML,2.604,20,100,40.0,0.0277,68.0,7200.0,1441.0,601280,22.0
3,2023-08-20 00:00:00,601281,6,5,N,ML,1.917,20,100,39.0,0.0214,68.1,7200.0,1441.0,601281,11.0
4,2023-08-20 00:00:00,601284,6,5,N,ML,2.988,20,100,37.0,0.0227,68.7,7200.0,1441.0,601284,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28219,2023-08-15 23:55:00,601357,6,5,N,ML,1.827,16,100,44.0,0.0508,60.1,1435.0,288.0,601357,12.0
28220,2023-08-15 23:55:00,601392,6,5,N,ML,2.280,16,100,27.0,0.0291,68.7,1435.0,288.0,601392,25.0
28221,2023-08-15 23:55:00,601510,6,5,N,ML,1.941,35,100,69.0,0.0363,69.8,1435.0,288.0,601510,1.0
28222,2023-08-15 23:55:00,601516,6,5,N,ML,3.669,16,100,42.0,0.0456,65.8,1435.0,288.0,601516,18.0


## Final Data Cleaning

In [33]:
"""
Create a table with timestep and PM (posted Miles) as one-one match
"""
timestep_value = [i for i in range(1, Occupancy['TimeStep'].max() + 1) for _ in range(Occupancy['Station_PM'].max() + 1)]
PM_values = list(range(Occupancy['Station_PM'].max() + 1)) * Occupancy['TimeStep'].max()
Time_PM = pd.DataFrame({'TimeStep': timestep_value, 'Station_PM': PM_values})

In [34]:
Occupancy_all = pd.merge(Time_PM, Occupancy, on=['TimeStep', 'Station_PM'] , how='left')
Flow_all = pd.merge(Time_PM, Flow, on=['TimeStep', 'Station_PM'] , how='left')
Speed_all = pd.merge(Time_PM, Speed, on=['TimeStep', 'Station_PM'] , how='left')

In [36]:
Occupancy_all_wide = Occupancy_all.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Flow_all_wide = Flow_all.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Speed_all_wide = Speed_all.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [37]:
Flow_wide = Flow.pivot(index='Station_PM', columns='TimeStep', values='Total_Flow')
Occupancy_wide = Occupancy.pivot(index='Station_PM', columns='TimeStep', values='Occupancy')
Speed_wide = Speed.pivot(index='Station_PM', columns='TimeStep', values='Speed')

In [38]:
Flow_all_wide.to_csv('Data_Clean/Flow_all.csv')
Occupancy_all_wide.to_csv('Data_Clean/Occupancy_all.csv')
Speed_all_wide.to_csv('Data_Clean/Speed_all.csv')

In [39]:
Flow_wide.to_csv('Data_Clean/Flow.csv')
Occupancy_wide.to_csv('Data_Clean/Occupancy.csv')
Speed_wide.to_csv('Data_Clean/Speed.csv')

In [40]:
Flow.to_csv('Data_Clean/Flow_Long.csv', index=False)
Occupancy.to_csv('Data_Clean/Occupancy_Long.csv', index=False)
Speed.to_csv('Data_Clean/Speed_Long.csv', index=False)

In [41]:
Flow_all.to_csv('Data_Clean/Flow_all_Long.csv', index=False)
Occupancy_all.to_csv('Data_Clean/Occupancy_all_Long.csv', index=False)
Speed_all.to_csv('Data_Clean/Speed_all_Long.csv', index=False)